In [1]:
import scipy.io as sio

In [2]:
import numpy as np

In [3]:
import cv2

In [4]:
from matplotlib import pyplot as plt

In [5]:
import random

In [6]:
MPII_MAT = r"/home/ubuntu/ShareFiles/ShareFiles/mpii_human_pose_v1/mpii_human_pose_v1_u12_2/mpii_human_pose_v1_u12_1.mat"

In [7]:
MPII_IMG = r"/home/ubuntu/PyTorch-YOLOv3/data/custom/images"

In [8]:
T=sio.loadmat(MPII_MAT, squeeze_me=True, struct_as_record=False)

In [9]:
M=T['RELEASE']

In [10]:
annots=M.annolist

In [11]:
is_train=M.img_train

In [12]:
def get_limb_label(point_list, point_select, head_length):
    p_ = []
    for i in point_select:
        for point in point_list:
            if i == point[0] and point[-1] == 1:
                p_.append([point[1],point[2]])
    if len(p_) == len(point_select):
        #find minimum and maximum
        np_array_ = np.asarray(p_).T
        x_max, y_max = np.amax(np_array_,1)[0], np.amax(np_array_,1)[1]
        x_min, y_min = np.amin(np_array_,1)[0], np.amin(np_array_,1)[1]
        length_half = head_length / 3
        x_max += length_half
        y_max += length_half
        x_min -= length_half
        y_min -= length_half
        w = x_max - x_min
        h = y_max - y_min
        x_c = x_min + w / 2
        y_c = y_min + h / 2
        return x_c, y_c, w, h, 1
    else:
        return 0,0,0,0,0

In [13]:
#shuffle the annots
random.shuffle(annots)

In [14]:
labels_dict = {}
for annot_id, annot in enumerate(annots):
    is_train_flag = is_train[annot_id]
    if is_train_flag == 1:#train:1 test:0.  18079 pics for train 6908 for test
        img_name = annot.image.name
        try:#train set has three pics disappeared
            pic=plt.imread(MPII_IMG+"/"+img_name)
        except:
            continue
        img_height = pic.shape[0]
        img_width = pic.shape[1]
        if 'annorect' in dir(annot):
            rects = annot.annorect
            if isinstance(rects, sio.matlab.mio5_params.mat_struct):
                rects = np.array([rects])
                labels_list = []#store labels which includes in one img
                for rect in rects:# one img may includes many person(rects)
                    try:
                        head_x1 = rect.x1
                        head_y1 = rect.y1
                        head_x2 = rect.x2
                        head_y2 = rect.y2
                        try:
                            points = rect.annopoints.point
                        except:
                            continue
                    except:
                        continue
                    head_length = head_x2 - head_x1
                    point_list = []
                    for point in points:
                        if point.is_visible in [0,1]:
                            is_visible = point.is_visible
                        else:
                            is_visible = 0
                        point_list.append([point.id,point.x,point.y,is_visible])
                    #get limbs labels
                    limbs_list = [[0,1],[1,2],[3,4],[4,5],[10,11],[11,12],[13,14],[14,15]]
                    lower_legs_list = [[0,1],[4,5]]
                    upper_legs_list = [[1,2],[3,4]]
                    lower_arms_list = [[10,11],[14,15]]
                    upper_arms_list = [[11,12],[13,14]]
                    for lower_legs in lower_legs_list:
                        x_c, y_c, w, h, flag = get_limb_label(point_list, lower_legs, head_length)
                        if flag:
                            xc = x_c / img_width
                            yc = y_c / img_height
                            wo = w / img_width
                            ho = h / img_height
                            cls = 1
                            labels_list.append([cls,xc,yc,wo,ho])
#                             cv2.rectangle(pic, (int(x_c-w/2),int(y_c-h/2)), (int(x_c+w/2),int(y_c+h/2)), (0,0,255),3)
#                             plt.imshow(pic)
#                             plt.show()
                    for upper_legs in upper_legs_list:
                        x_c, y_c, w, h, flag = get_limb_label(point_list, upper_legs, head_length)
                        if flag:
                            xc = x_c / img_width
                            yc = y_c / img_height
                            wo = w / img_width
                            ho = h / img_height
                            cls = 2
                            labels_list.append([cls,xc,yc,wo,ho])
#                             cv2.rectangle(pic, (int(x_c-w/2),int(y_c-h/2)), (int(x_c+w/2),int(y_c+h/2)), (255,0,0),3)
#                             plt.imshow(pic)
#                             plt.show()
                    for lower_arms in lower_arms_list:
                        x_c, y_c, w, h, flag = get_limb_label(point_list, lower_arms, head_length)
                        if flag:
                            xc = x_c / img_width
                            yc = y_c / img_height
                            wo = w / img_width
                            ho = h / img_height
                            cls = 3
                            labels_list.append([cls,xc,yc,wo,ho])
#                             cv2.rectangle(pic, (int(x_c-w/2),int(y_c-h/2)), (int(x_c+w/2),int(y_c+h/2)), (0,255,0),3)
#                             plt.imshow(pic)
#                             plt.show()
                    for upper_arms in upper_arms_list:
                        x_c, y_c, w, h, flag = get_limb_label(point_list, upper_arms, head_length)
                        if flag:
                            xc = x_c / img_width
                            yc = y_c / img_height
                            wo = w / img_width
                            ho = h / img_height
                            cls = 4
                            labels_list.append([cls,xc,yc,wo,ho])
#                             cv2.rectangle(pic, (int(x_c-w/2),int(y_c-h/2)), (int(x_c+w/2),int(y_c+h/2)), (255,0,255),3)
#                             plt.imshow(pic)
#                             plt.show()
                    #head
                    w = head_x2 - head_x1
                    h = head_y2 - head_y1
                    xc = (head_x1 + w / 2) / img_width
                    yc = (head_y1 + h / 2) / img_height
                    wo = w / img_width
                    ho = h / img_height
                    cls = 0
                    labels_list.append([cls,xc,yc,wo,ho])
#                     cv2.rectangle(pic, (int(head_x1),int(head_y1)), (int(head_x2),int(head_y2)), (128,90,255),3)
#                     plt.imshow(pic)
#                     plt.show()
#                     for limb in limbs_list:
#                         x_c, y_c, w, h, flag = get_limb_label(point_list, limb, head_length)
#                         if flag:
#                             xc = x_c / img_width
#                             yc = y_c / img_height
#                             wo = w / img_width
#                             ho = h / img_height
#                             cls = 0
#                             labels_list.append([cls,xc,yc,wo,ho])
#                             print([x_c, y_c, w, h])
#                             cv2.rectangle(pic, (int(x_c-w/2),int(y_c-h/2)), (int(x_c+w/2),int(y_c+h/2)), (0,0,255),3)
#                             plt.imshow(pic)
#                             plt.show()
                if labels_list:
                    labels_dict[img_name] = labels_list
                        

/home/ubuntu/anaconda3/envs/pytorch-yolov3/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


In [16]:
IMG_train_txt = r"./train.txt"
IMG_valid_txt = r"./valid.txt"
# LABEL_train = r"./train"
# LABEL_valid = r"./valid"
LABEL=r"./labels"
# LABEL=r"/home/ubuntu/PyTorch-YOLOv3/data/custom/labels"

In [17]:
train_txt = open(IMG_train_txt,'a')
valid_txt = open(IMG_valid_txt,'a')

In [18]:
def write_label(folder, name, text):
    txt = open(folder+'/'+name[:-4]+'.txt','a')
    for n in text:
        wt_str = ""
        for item_ in n:
            wt_str = wt_str + str(item_) + " "
        txt.write(wt_str[:-1]+"\r")
    txt.close()

In [20]:
for_train = 0
len_data = len(labels_dict)
for img_name, labels_list in labels_dict.items():
    if for_train < len_data*0.8:
        #open train.txt to store the img path
        train_txt.write(MPII_IMG+"/"+img_name+"\r")
        for_train += 1
        write_label(LABEL, img_name, labels_list)
    else:
        #open val.txt to store the img path
        valid_txt.write(MPII_IMG+"/"+img_name+"\r")
        write_label(LABEL, img_name, labels_list)
train_txt.close()
valid_txt.close()
